In [14]:
%run 01_hash.ipynb

In [15]:
import json
from copy import deepcopy

Print objects in a nice way.

In [16]:
def stringfy(o):
    s = []
    for k,v in o.__dict__.items():
        p = f'{k}:'.ljust(15)
        if hasattr(v,'messageHash'): s.append(p+ph(v.messageHash.hex()))
        elif str(v)[:2]=='0x'      : s.append(p+ph(v))
        elif type(v)   ==float     : s.append(p+str(round(v,8))+' eth')
        else                       : s.append(p+str(v))
    return '\n'.join(s)

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `fee` goes to the miner who included the tx in a block. More on mining in [03_block.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/04_mine.ipynb). The higher it is, the more likely it is included in a block.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation, see [here](https://kb.myetherwallet.com/en/transactions/what-is-nonce/). 

It's `hash` is a unique fingerprint. Every time something in the tx changes, it gets recalculated.

In [17]:
class TX(Hashable): 
    def __init__(self, fr, to, value, fee, nonce): 
        self.fr, self.to = fr, to
        self.value       = float(value)
        self.fee         = fee
        self.nonce       = nonce
        self.time        = time.ctime()
        self.signed      = False
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop not in ['sig','signed','hash']: 
            super().__setattr__('hash', sha(self.__dict__))
        if prop == 'sig': self.signed = True
    
    def json(self): return json.dumps({**self.__dict__}, indent=4)
    def smry(self): return f'{pmh(self.fr)} -> {pmh(self.to)} {self.value} eth'
    def __str__(self): return stringfy(self)

Create random txs.

In [18]:
tx1 = TX(rh(), rh(), 12, 0.2,  0)
tx2 = TX(rh(), rh(), 6,  0.15, 0); print(tx1)

fr:            📴 0x8e41f366c8...043
hash:          📃 0x5df4e68bc1...50d
to:            📱 0x8b813cc5ae...b88
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Mon Apr  5 23:30:06 2021
signed:        False


#### TX Summary

In [22]:
tx1.smry()

'📴 0x8e41 -> 📱 0x8b81 12.0 eth'

### TX JSON

Every tx has a json representation that we will use for our API later.

In [23]:
tx1_json = tx1.json(); print(tx1_json)

{
    "fr": "0x8e41f366c864d8c909839ab9b7c5f17e2c46ba280ae0538e9befd4b860797043",
    "hash": "0x5df4e68bc118a2cb9866322ca588347e62f1f66652dc85793608d609673bd50d",
    "to": "0x8b813cc5ae1093377a2f8d954df8d7e1989de0ba32ca4f6ce5ba583361017b88",
    "value": 12.0,
    "fee": 0.2,
    "nonce": 0,
    "time": "Mon Apr  5 23:30:06 2021",
    "signed": false
}


We can load this JSON string `d` and create a tx object.

In [24]:
def load_tx(d):
    d = json.loads(d)
    tx = TX(d['fr'],d['to'],d['value'],d['fee'],d['nonce'])
    for k,v in d.items(): setattr(tx, k, v)
    tx.hash = d['hash']
    return tx

In [25]:
tx1_from_json = load_tx(tx1_json)
assert tx1 == tx1_from_json

#### Print several txs in a nice way.

In [26]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])
print(txs2str([tx1,tx2]))

fr:            📴 0x8e41f366c8...043
hash:          📃 0x5df4e68bc1...50d
to:            📱 0x8b813cc5ae...b88
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Mon Apr  5 23:30:06 2021
signed:        False

fr:            📁 0x5b543684db...c0b
hash:          🔲 0xcca6d2c5f7...3ab
to:            📎 0x6851299a4e...029
value:         6.0 eth
fee:           0.15 eth
nonce:         0
time:          Mon Apr  5 23:30:06 2021
signed:        False



#### Changing Tx
Every change in the object is reflected by its hash. Compare the tx hash below with the hash of unchanged tx above. The tx hash has changed. This is how we can make sure that nobody changes the tx without us knowing.

In [27]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2_false_value)

fr:            📁 0x5b543684db...c0b
hash:          🕎 0xe8424d270b...fce
to:            📎 0x6851299a4e...029
value:         120
fee:           0.15 eth
nonce:         0
time:          Mon Apr  5 23:30:06 2021
signed:        False


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [28]:
assert tx2 != tx2_false_value

#### Signing Tx
An account (implemented in [02_account.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/02_account.ipynb)) has the ability to sign a tx. Here we simply mock a signature with a random hash. The signature `sig` is saved as an attribute.

In [29]:
tx1_signed = deepcopy(tx1)
tx1.sig    = rh(); print(tx1)

fr:            📴 0x8e41f366c8...043
hash:          📃 0x5df4e68bc1...50d
to:            📱 0x8b813cc5ae...b88
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Mon Apr  5 23:30:06 2021
signed:        True
sig:           📥 0x7f1eb5b010...128


Signing the tx should not change its hash. Therefore it is equal to the unsigned tx.

In [30]:
assert tx1 == tx1_signed